In [1]:
import tensorflow as tf
import os 
from PIL import Image
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from nets import nets_factory
from preprocessing import preprocess_image
from IPython.display import clear_output
slim =tf.contrib.slim

SPLITS_TO_SIZES = {'train': 12936, 'query':3800, 'test': 10000}
_ITEMS_TO_DESCRIPTIONS = {
    'img_raw': 'A [128 x 64 x 3] color image.',
    'label': 'A single integer between 0 and 9',
}
_NUM_CLASSES = 751

In [3]:

def input_fn():
    dataset = tf.contrib.data.TFRecordDataset(['/tmp/Market-1501/market-1501_query.tfrecord'])

    # Use `tf.parse_single_example()` to extract data from a `tf.Example`
    # protocol buffer, and perform any additional per-record preprocessing.
    def _parser(record):
        keys_to_features = {
            "img_raw": tf.FixedLenFeature((), tf.string, default_value=""),
            "label": tf.FixedLenFeature((), tf.int64,default_value=0),
            "cam": tf.FixedLenFeature((), tf.int64,default_value=0)
            }
        parsed = tf.parse_single_example(record, keys_to_features)

        # Perform additional preprocessing on the parsed data.
        image = tf.decode_raw(parsed["img_raw"],tf.uint8)
        image = tf.reshape(image, [128, 64, 3])
#         image = tf.cast(image,tf.float32)
        image = preprocess_image(image,224,224,is_training=True)
        label = tf.cast(parsed["label"], tf.int32)
        cam = tf.cast(parsed["cam"], tf.int32)
        return image, label, cam

    # Use `Dataset.map()` to build a pair of a feature dictionary and a label
    # tensor for each example.
    dataset = dataset.map(_parser)
    dataset = dataset.repeat()
    dataset = dataset.batch(32)
#     if is_training:
#         dataset = dataset.shuffle(buffer_size=10000)
    iterator = dataset.make_one_shot_iterator()

    # `features` is a dictionary in which each value is a batch of values for
    # that feature; `labels` is a batch of labels.
    
    imgs, labels, cams = iterator.get_next()
    return imgs, labels, cams


def get_restore_variabels():
    exclusions = []
    
    checkpoint_exclude_scopes=["resnet_v1_50/logit"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    # TODO(sguada) variables.filter_variables()
    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)

    return variables_to_restore


def initialize_uninitialized_vars(sess):
    from itertools import compress
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))

    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))

        
 

In [ ]:

with tf.Graph().as_default() as graph:

    images,labels,cam = input_fn()
    labels = tf.contrib.layers.one_hot_encoding(labels,751)
    network_fn = nets_factory.get_network_fn('resnet_v1_50',num_classes=751)
    logits, _ = network_fn(images)
    variables_to_restore = get_restore_variabels()
    saver = tf.train.Saver(variables_to_restore,max_to_keep=4)
    
    with tf.Session() as sess:
#         image = graph.get_tensor_by_name('reshape_7:0')

        saver.restore(sess, '/tmp/checkpoints/market-1501/pretrain/resnet_v1_50.ckpt')

        tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=labels)
        total_loss = tf.losses.get_total_loss()
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(total_loss,global_step=tf.train.get_global_step())

        initialize_uninitialized_vars(sess)
        for i in range(18000):
#         while True:
            try:
#                 im,ll = sess.run([images,labels])
                _,loss = sess.run([train_op,total_loss])
#                 lls += [ll]
                print("step: %s , loss: %s"%(i,loss))
                if i%20 == 0:
                    clear_output()
                if i % 2000 ==0:
                    saver.save(sess,'/tmp/checkpoints/market-1501/resnet_v1_50/',global_step=i)
            except tf.errors.OutOfRangeError:
                break
np.array(lls).shape

step: 1121 , loss: 6.60209
step: 1122 , loss: 6.60681
step: 1123 , loss: 6.62193
step: 1124 , loss: 6.6069
step: 1125 , loss: 6.63948
step: 1126 , loss: 6.61751


()

# Check graph operations 

In [ ]:
# softmax_tensor = endpoints['resnet_v1_50/block4/unit_3/bottleneck_v1']
softmax_tensor = sess.graph.get_tensor_by_name('resnet_v1_50/Pad/input:0')
softmax_tensor
# tf.GraphKeys.GLOBAL_VARIABLES
# sess.graph.get_all_collection_keys()
# for op in sess.graph.get_operations():
#     print( op.name)

# tf.get_collection('model_variables')

# Extract features

In [ ]:

feature_map = {
                'vgg_16': 'fc7_end_points',
                'inception_v3':'PreLogits',
                'inception_v4': 'PreLogitsFlatten',
                'inception_resnet_v2': 'PreLogitsFlatten',
                'resnet_v1_50': 'pool5_end_points',
                'resnet_v2_50': 'pool5_end_points',
                'mobilenet_v1': 'AvgPool_1a',
               }


features = []
classes = []
cameras = []
record_file='/tmp/Market-1501/market-1501_query.tfrecord'
with tf.Graph().as_default():
    with tf.Session() as sess:
    #     init_op = tf.global_variables_initializer()
    #     sess.run(init_op)
    #     coord = tf.train.Coordinator()
    #     threads = tf.train.start_queue_runners(coord=coord)
        image,label,cam = input_fn(record_file)
        network_fn = nets_factory.get_network_fn('inception_v3',num_classes=751)
        train_image_size = network_fn.default_image_size
        image = preprocess_image(image,train_image_size,train_image_size)
        images,labels,cams = tf.train.batch([image,label,cam],batch_size=32)

        logits, endpoints = network_fn(images)
        
        
        
        for k in endpoints:
            print(k,'sssssss',endpoints[k])
    #     images,labels,cams = tf.train.batch([image,label,cam],batch_size=4)
    #     for i in range(3):
    # #     while True:
    #         try:
    #             aa,bb,cc = sess.run([image,label,cam])
    #             features += [aa]
    #             classes += [bb]
    #             cameras += [cc]
    #         except tf.errors.OutOfRangeError:
    #             break
    #     coord.request_stop()

    #     coord.join(threads)
